In [19]:
from astropy.io import fits
from matplotlib import pyplot
import scipy.optimize as opt
import numpy as np
from pathlib import Path
import re
import os
%matplotlib inline

In [20]:
p = Path("/scratch/datasets/astro_deconv_2019/")
output = Path("/scratch/datasets/astro_deconv_2019_cleanbeam_better")

In [11]:
psf_file = list(p.glob("*-psf.fits"))[0]

In [12]:
data = fits.open(psf_file)[0].data.squeeze()
s = data.shape[0]
midpixel = s // 2

In [17]:
# todo landman: dont fit for amplitude, xo and xy, only sigma_x and sigma_y
def twoD_Gaussian(pos, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    x, y = pos
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

def fitted(pos, sx, sy, t, o):
    return twoD_Gaussian(pos, amplitude=1, xo=midpixel, yo=midpixel,
                         sigma_x=sx, sigma_y=sy, theta=t, offset=o)

In [14]:
def fit(data):
    s = data.shape[0]
    
    x = np.linspace(0, s-1, s)
    y = np.linspace(0, s-1, s)
    x, y = np.meshgrid(x, y)
    initial_guess = (1, s/2, s/2, 20, 20, 0, 0 )
    popt, pcov = opt.curve_fit(f=twoD_Gaussian, xdata=(x, y), ydata=data.flatten(), p0=initial_guess,  maxfev=10000)
    return twoD_Gaussian((x, y), *popt).reshape(s, s)

In [15]:
def fit2(data):
    x = np.linspace(0, s-1, s)
    y = np.linspace(0, s-1, s)
    x, y = np.meshgrid(x, y)
    initial_guess = (1, 1, 0, 0)
    popt, pcov = opt.curve_fit(f=fitted, xdata=(x, y), ydata=data.flatten(), p0=initial_guess,  maxfev=10000)
    return fitted((x, y), *popt).reshape(s, s)

In [24]:
if not output.is_dir():
    output.mkdir()
    
for psf_file in list(p.glob("*-wsclean-psf.fits")):

    index = re.match(r'(\d+)-wsclean-psf\.fits', pathlib.Path(psf_file).name)[1]
    target = f'{output}/{index}-clean-beam.fits'
    if pathlib.Path(target).is_file():
        print('skipping ' + target)
        continue
    print(psf_file)
    f = fits.open(psf_file)
    data = f[0].data.squeeze()
    clean_beam = fit2(data)
    hdu = fits.PrimaryHDU(clean_beam)
    hdu.header = f[0].header
    hdul = fits.HDUList([hdu])
    hdu.header = f[0].header
    hdul.writeto(target, overwrite=True)


/scratch/datasets/astro_deconv_2019/8169-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7543-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5101-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1003-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9858-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4144-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5598-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2506-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7677-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1467-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4050-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5039-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8858-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9374-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/711-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5672-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2099-wsclean-psf.fits
/scratch/datase

/scratch/datasets/astro_deconv_2019/1314-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2028-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9832-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9796-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8370-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5706-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7512-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2901-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8569-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7227-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6765-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6597-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2094-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8030-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/769-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7137-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4494-wsclean-psf.fits
/scratch/datase

/scratch/datasets/astro_deconv_2019/511-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2393-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1939-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2656-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1177-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6479-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7942-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7465-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/3745-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4301-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2602-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/3164-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4498-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1770-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2238-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/923-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/3185-wsclean-psf.fits
/scratch/dataset

/scratch/datasets/astro_deconv_2019/3576-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7495-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/391-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1976-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2693-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7178-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/338-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6691-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2739-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1150-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/3540-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9158-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9508-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4740-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7122-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/3307-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4344-wsclean-psf.fits
/scratch/dataset

/scratch/datasets/astro_deconv_2019/9615-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/612-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/319-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9781-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4580-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8980-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/969-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4701-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/3389-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8962-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5503-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9186-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5786-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4284-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/3218-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4988-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2676-wsclean-psf.fits
/scratch/datasets

...

/scratch/datasets/astro_deconv_2019/5353-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7927-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8217-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8116-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5018-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2827-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1038-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2206-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/3140-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1973-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7161-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6681-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/188-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6064-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6463-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9472-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6516-wsclean-psf.fits
/scratch/datase

/scratch/datasets/astro_deconv_2019/52-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2576-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8002-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6170-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9114-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9315-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6800-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5305-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5788-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4453-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7018-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8452-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5501-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1346-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9920-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1336-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/521-wsclean-psf.fits
/scratch/datasets

/scratch/datasets/astro_deconv_2019/1126-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2536-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6036-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2822-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6125-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/457-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8114-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8359-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6042-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7172-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4982-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4669-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2050-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9578-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/8874-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2543-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4589-wsclean-psf.fits
/scratch/datase

/scratch/datasets/astro_deconv_2019/2205-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2179-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7519-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1584-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5399-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4292-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1734-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2625-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1059-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9177-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/208-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5196-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2027-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2671-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1454-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/2376-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5709-wsclean-psf.fits
/scratch/datase

/scratch/datasets/astro_deconv_2019/1610-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4948-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7994-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/4546-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5055-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9911-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6549-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6350-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7771-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1776-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/9425-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/1671-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/3827-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/5819-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7455-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/6550-wsclean-psf.fits
/scratch/datasets/astro_deconv_2019/7687-wsclean-psf.fits
/scratch/datas

In [ ]:
_ = pyplot.imshow(clean_beam)

In [ ]:
_ = pyplot.imshow(data)